# 역전파 학습법

## 심층 신경망의 구조

 - 네트워크 구조를 좀 더 복잡하게 구성
 - 얕은 신경망(SNN)보다 은닉계층이 많은 신경망을 DNN이라고 부른다.
 - 보통 5개 이상의 계층이 있는 경우 '깊다'(Deep)라고 표현

## 무엇이 다를까
 - 은닉 계층 추가 = 특징의 비선형 변환 추가
 - 학습 매개변수의 수가 계층 크기의 제곱에 비례
 - Sigmoid 활성 함수 동작이 원할하지 않음
   - ReLu(Rectified Linear Unit)도입 필요

## 역전파 학습법의 개념
 - 학습 환경이 주어졌을 때, 손실 함수를 매개 변수로 여러 번 미분해야 한다.
 - 의존성이 있는 함수의 계산
    - 동일 연산이 2회 필요하므로, 중복되는 계산이 1회 발생한다.
 - 의존성이 있는 함수의 계산의 문제를 해결하기 위해 : 동적 계획법(Dynamic Programming)
    - 처음 계산될 때 값은 한번 저장, 첫 계산시 값을 저장하므로 중복 계산이 발생하지 않는다.
    - 하게될 미분 연산은 동일한 연산 값을 여러번 참조해야하기 때문에 동적 계획법이 효율적
 - 연쇄 법칙 Chain Rule
    - 연속된 두 함수의 미분은, 각 함수의 미분을 연쇄적으로 곱한 것과 같다.
 - 출력 계층의 미분 : 연쇄 법칙을 이용하려면 손실 함수의 미분이 필요하다
 - 마지막 은닉 계층의 미분 : 연쇄 법칙을 이용하려면 손실함수, 출력계층의 미분이 필요하다. 출력 계층, 손실함수의 미분을 저장해 두면(동적계획법) 중복 연산을 피할 수 있다.
 - 연쇄 법칙을 이용하려면 손실함수, 출력계층, 사이의 모든 은닉계층의 미분이 필요하다.
 
### 순방향 추론 Forward Inference
 - 현재 매개변수에서 손실 값을 계산하기 위해 순차적ㅇ니 연산을 수행하는 것을 순방향 추론이라 한다.
 - 학습을 마친 후 알고리즘을 사용할 때는 순방향 추론을 사용한다.

### 역전파 학습법 Back-Propagation
 - 심층 신경망의 미분을 계산하기 위해, 연쇄 법칙과 동적 계획법을 이용하여 효율적으로 계산할 수 있다. 

## 역전파 학습의 필요성
 - 블랙박스 모델의 학습
    - 매개변수에 따라 동작이 달라진다.
 - 수치적 기울기 Numerical Gradient : 미분의 정의로부터 극한 연산을 근사해 수치적 기울기를 구할 수 있다.
 - 블랙박스 모델의 수치적 기울기
    - 각 스칼라 변수를 각각 조금씩 바꾸어 대입해 보면서 수치적 기울기를 구한다.
 - 심층 신경망의 수치적 기울기
    - 10만개 파라미터를 가진 경우 무려 100억회를 곱해야한다. 
    - 이를 해결하기 위해 역전파 학습을 한다

## 합성 함수와 연쇄법칙
 - 연쇄 법칙을 이용하면 연속된 함수의 미분을 각각의 미분의 곱으로 표현할 수 있다.
 - f+n(h_n-1) = a(W_nh_n-1 _ b_n)

### 이미 손실을 구했다면, 데이터의 입력과 출력은 학습 과정에서 중요하지 않다.
 - 손실을 최소화하는 파라미터만 찾으면 되기 때문!
 - 미분하고자하는 경로사이에 있는 모든 미분 값을 곱하면 원하는 미분을 구할 수 있다.
    - 즉, 원하는 미분을 얻으려면 경로 사이에 있는 모든 미분 값을 다 알아야 한다는 말이다.
    
### Sigmoid 함수의 미분
 - 초창기 신경망에 가장 많이 쓰인 Sigmoid 활성 함수의 미분. 정리된 결과를 이용하면 매우 간단하게 미분할 수 있다.
 
## 역전파 알고리즘
 - 오류 역전파 알고리즘(Back-Propagation Algorithm; BP)
 - 정방향 연산 시, 계측별로 BP에 필요한 중간 결과를 저장한다.
 - Loos를 각 파라미터로 미분한다. 연쇄법칙(역방향 연산)을 이용한다.
 - 미분의 연쇄 법칙과 각 함수의 수식적 미분을 이용하면, 단 한번의 손실 함수 평가로 미분을 구할 수 있다. 단, 중간 결과를 저장해야 하므로 메모리를 추가로 사용한다.

# 수치 미분을 이용한 심층 신경망 학습

In [31]:
import time
import numpy as np

# 유틸리티 함수

In [32]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

# 뉴런 구현

In [33]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameter
        self.W = W
        self.b = b
        self.a = a
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        
    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b) # activation((W^T)x + b)
    

# 심층 신경망 구현

In [37]:
class DNN:
    # hidden_depth(히든 레이어 갯수), num_neuron(히든 레이어 하나당 뉴런이 몇개 있는지) 
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))
        
        self.sequence = list()
        #First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))
        
        #Hidden layers
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))
        
        #Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))
        
    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_gradient(self, x, y, loss_func): # x(추정), y(정답)
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence): # iterate layer
            for w_i, w in enumerate(layer.W): #iterate W (row)
                for w_j, ww in enumerate(w): #iterate W (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron = Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon # (f(x+eps) - f(x)) / epsilon
                    layer.dW[w_i][w_j] = num_grad
                    
                for b_i, bb in enumerate(layer.b): # iterate b
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron = Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon # (f(x+eps) - f(x)) / epsilon
                    layer.db[b_i] = num_grad
        return loss

# 경사하강 학습법

In [38]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

# 동작 테스트

In [ ]:
# 임의의 데이터 셋
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.27381181131232146
Epoch 1: Test loss 0.2722886780345872
Epoch 2: Test loss 0.2707744287998881
Epoch 3: Test loss 0.2692691172362483
Epoch 4: Test loss 0.26777279378142216
Epoch 5: Test loss 0.2662855057134419
Epoch 6: Test loss 0.26480729718330176
Epoch 7: Test loss 0.26333820924968077
Epoch 8: Test loss 0.26187827991572454
Epoch 9: Test loss 0.26042754416752173
Epoch 10: Test loss 0.2589860340144733
Epoch 11: Test loss 0.257553778531255
Epoch 12: Test loss 0.2561308039013556
Epoch 13: Test loss 0.254717133462038
Epoch 14: Test loss 0.25331278775061383
Epoch 15: Test loss 0.2519177845520455
Epoch 16: Test loss 0.25053213894760185
Epoch 17: Test loss 0.2491558633646338
Epoch 18: Test loss 0.24778896762722719
Epoch 19: Test loss 0.24643145900780505
Epoch 20: Test loss 0.24508334227939182
Epoch 21: Test loss 0.2437446197686337
Epoch 22: Test loss 0.24241529140932716
Epoch 23: Test loss 0.2410953547965326
Epoch 24: Test loss 0.2397848052410272
Epoch 25: Test loss 0.238